## Agent

에이전트 프레임워크를 가장 잘 이해하기 위해서, 온라인에서 정보를 찾는 도구 하나와 우리가 인덱스로 로드한 특정 데이터를 찾는 도구 하나를 가진 에이전트를 만들어 보겠습니다.

### 단계별 추적을 위한 LangSmith 설정

LangChain으로 구축한 애플리케이션은 여러 단계에 걸쳐 LLM 호출을 여러 번 사용하게 됩니다.

단계별로 호출되는 로직이 더 복잡해짐에 따라, 체인이나 에이전트 내부에서 정확히 무슨 일이 일어나고 있는지 조사할 수 있는 능력이 매우 중요해집니다. 이를 위한 최선의 방법은 [LangSmith](https://smith.langchain.com)를 사용하는 것입니다.

LangSmith가 필수는 아니지만, 매우 유용한 도구입니다. LangSmith를 사용하고 싶다면, 위의 링크에서 가입한 후, 로깅 추적을 시작하기 위해 환경 변수를 설정해야 합니다.

## 도구(Tools)

Agent 가 활용할 도구를 정의하여 Agent 가 추론(reasoning)을 수행할 때 활용하도록 만들 수 있습니다.

Tavily Search 는 그 중 대표적인 **검색 도구** 입니다. 검색을 통해 최신 정보에 접근하여 검색 결과를 가지고 답변을 생성할 수 있습니다. 도구는 이처럼 검색 도구 뿐만아니라 Python 코드를 실행할 수 있는 도구, 직접 정의한 함수를 실행하는 도구 등 다양한 종류와 방법론을 제공합니다.


### 검색도구: Tavily Search

LangChain에는 Tavily 검색 엔진을 도구로 쉽게 사용할 수 있는 내장 도구가 있습니다.

Tavily Search 를 사용하기 위해서는 API KEY를 발급 받아야 합니다.

- [Tavily Search API 발급받기](https://app.tavily.com/sign-in)

발급 받은 API KEY 를 다음과 같이 환경변수에 등록 합니다.

아래 코드의 주석을 풀고 발급받은 API KEY 를 설정합니다.


In [1]:
import os

# TAVILY API KEY를 기입합니다.
# os.environ["TAVILY_API_KEY"] = "TAVILY API KEY 입력"

# 디버깅을 위한 프로젝트명을 기입합니다.
os.environ["LANGCHAIN_PROJECT"] = "AGENT TUTORIAL"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
# TavilySearchResults 클래스를 langchain_community.tools.tavily_search 모듈에서 가져옵니다.
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=5은 검색 결과를 5개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=5)

`search.invoke` 함수는 주어진 문자열에 대한 검색을 실행합니다.

`invoke()` 함수에 검색하고 싶은 검색어를 넣어 검색을 수행합니다.


In [4]:
# 검색 결과를 가져옵니다.
search.invoke("판교 몽중헌 전화번호는 무엇인가요?")

[{'url': 'https://app.catchtable.co.kr/ct/shop/mongjungheon_pg',
  'content': '중식 | 판교 몽중헌 판교점 ... - 판교 테크원 타워 건물 내 지하 주차장을 이용 부탁드리며 , 2시간 이용가능 합니다. 주류 및 콜키지 안내 ... 전화번호 031-601-7574 ...'},
 {'url': 'https://blog.naver.com/PostView.naver?blogId=pdi0903&logNo=222685325353&directAccess=false',
  'content': "ㅋㅋ\n\u200b\n메인 메뉴판 보실게요\n\u200b\n\u200b\n사진을 찍다가 지쳐하니까\n앞에 일행이 메뉴판을 들어줬습니다\nㅋㅋㅋㅋㅋ\n민망..\n\u200b\n저희 일행은\n딤섬 4종류\u200b\n하교(통새우) ₩12,000\n구채교(새우와 부추) ₩12,000\n근채교(새우, 돼지고기, 샐러리, 칠리페퍼) ₩12,000\n상하이소롱포(상하이식 돼지고기) ₩15,000\n그리고\n삼선짬뽕 ₩18,000\n사천탕면 ₩18,000\n게살볶음밥 ₩18,000\n을 주문했습니다.\n 왜 딤섬전문점인지 알겠다는 ㅎㅎ\n\u200b\n이어서 나온\n\u200b\n\u200b\n일행1의 XO소스 게살 볶음밥\n\u200b\n\u200b\n일행2의 사천탕면\n\u200b\n\u200b\n이건 제가 주문한 삼선짬뽕 입니다.\n \u200b\n몽중헌 판교점\n경기 성남시 분당구 분당내곡로 131, 판교테크원타워 2층\n매장전화번호) 031-601-7574\n영업시간) 11:30 ~ 22:00\n(브레이크타임 15:00 ~ 17:30)\n\u200b\n판교테크원타워는 판교역 1번출구로 나오면\n바로 보이는 이곳\n\u200b\n\u200b\n투썸플레이스가 있는 이 건물이 되겠습니다!!\n \u200b\n딤섬으로 유명한 '몽중헌'은\n판교역 바로앞에 있는\n판교테크원타워 2층에 위치해있구요\n몇주전부터 오픈예정 현수막을 보고\

### PDF 기반 문서 검색 도구: Retriever

우리의 데이터에 대해 조회를 수행할 retriever도 생성합니다.


이 코드는 웹 기반 문서 로더, 문서 분할기, 벡터 저장소, 그리고 OpenAI 임베딩을 사용하여 문서 검색 시스템을 구축합니다.

여기서는 PDF 문서를 `FAISS` DB 에 저장하고 조회하는 retriever 를 생성합니다.


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

이 함수는 `retriever` 객체의 `get_relevant_documents` 메소드를 사용하여 "YouTube의 2024년부터 AI 생성콘텐츠 표시 의무화에 대한 내용을 알려줘" 와 관련된 문서들 중 첫 번째 문서를 반환합니다.

이는 특정 쿼리에 대한 가장 **관련성 높은 문서** 를 찾는 데 사용됩니다.


In [6]:
# PDf 문서에서 Query 에 대한 관련성 높은 Chunk 를 가져옵니다.
retriever.get_relevant_documents(
    "YouTube의 2024년부터 AI 생성콘텐츠 표시 의무화에 대한 내용을 알려줘"
)[0]

/Users/teddy/Library/Caches/pypoetry/virtualenvs/langchain-kr-lwwSZlnu-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Document(page_content='SPRi AI Brief |  \n2023-12 월호\n14유튜브 , 2024년부터 AI 생성 콘텐츠 표시 의무화 \nn유튜브가 몇 달 안에 생성 AI를 사용한 콘텐츠에 AI 라벨 표시를 의무화하기로 했으며 , 이를 \n준수하지 않는 콘텐츠는 삭제하고 크리에이터에 대한 수익 배분도 중단할 수 있다고 설명\nn유튜브는 AI 생성 콘텐츠가 신원 파악이 가능한 개인을 모방한 경우 개인정보 침해 신고 \n절차에 따라 콘텐츠 삭제 요청도 받을 계획KEY Contents\n£유튜브 , 생성 AI 콘텐츠에 AI 라벨 표시 안 하면 콘텐츠 삭제\nn유튜브가 2023 년 11월 14일 공식 블로그를 통해 몇 달 안에 생성 AI를 사용한 콘텐츠에 AI \n라벨을 표시하는 새로운 규칙을 시행한다고 발표 \n∙실제로 일어나지 않은 사건을 사실적으로 묘사하거나 실제로 하지 않은 말이나 행동을 보여주는 콘텐츠와 \n같이 AI 도구를 사용해 사실적으로 변경되거나 합성된 콘텐츠에는 AI 라벨을 표시 필요\n∙유튜브는 이러한 규칙이 선거나 분쟁 상황, 공중 보건, 공직자 관련 문제와 같이 민감한 주제를 다루는 \n콘텐츠에서 특히 중요하다고 강조했으며 , 크리에이터가 AI로 제작한 콘텐츠에 AI 라벨을 표시하지 않으면 \n해당 콘텐츠는 삭제되고 광고 수익을 배분하는 유튜브 파트너 프로그램도 정지될 수 있음\n∙유튜브는 두 가지 방식으로 AI를 이용한 콘텐츠의 변경이나 합성 여부를 시청자에게 전달할 계획으로 \n동영상 설명 패널에 라벨을 표시하는 방식이 기본이며 , 민감한 주제를 다루는 특정 유형의 콘텐츠는 동영상 \n플레이어에 더욱 눈에 띄는 라벨을 적용 \n∙유튜브는 커뮤니티 정책에 위반되는 일부 합성 콘텐츠에 대해서는 라벨 지정 여부와 관계없이 삭제할 \n방침으로 , 가령 사실적인 폭력을 보여주는 합성 동영상이 시청자에게 충격이나 혐오감을 줄 수 있다면 \n삭제될 수 있음\n£유튜브 , 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 

이제 우리가 검색을 수행할 인덱스를 채웠으므로, 이를 에이전트가 제대로 사용할 수 있는 도구로 쉽게 변환할 수 있습니다.


`create_retriever_tool` 함수는 `langchain` 라이브러리의 `tools.retriever` 모듈에서 가져온다. 이 함수는 특정 데이터를 검색하기 위한 도구를 생성하는 데 사용된다. `langchain`은 언어 모델과 관련된 다양한 기능을 제공하는 라이브러리로, 이 중 검색 도구 생성 기능은 데이터 검색 및 처리 작업을 용이하게 한다.


In [7]:
# langchain 패키지의 tools 모듈에서 retriever 도구를 생성하는 함수를 가져옵니다.
from langchain.tools.retriever import create_retriever_tool

이 함수는 `create_retriever_tool`을 호출하여 `retriever_tool`을 생성합니다. 이 도구는 `LangSmith`에 대한 정보를 검색하는 데 사용됩니다.

사용자가 `LangSmith`에 대해 궁금한 점이 있을 때, 이 도구를 통해 질문을 해결할 수 있습니다. 함수는 `retriever` 인스턴스, 검색 도구의 이름인 `langsmith_search`, 그리고 사용자에게 도구 사용 방법을 안내하는 메시지를 인자로 받습니다.


In [8]:
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description="2023년 12월 AI 관련 정보를 PDF 문서에서 검색합니다. '2023년 12월 AI 산업동향' 과 관련된 질문은 이 도구를 사용해야 합니다!",
)

### Agent 가 사용할 도구 목록 정의

이제 두 가지를 모두 만들었으므로, Agent 가 사용할 도구 목록을 만들 수 있습니다.


`tools` 리스트는 `search`와 `retriever_tool`을 포함합니다. 이 리스트는 검색 및 정보 검색 도구를 저장하는 데 사용됩니다. 각 요소는 특정 작업을 수행하는 데 필요한 기능을 제공합니다.


In [9]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

## 에이전트 생성

이제 도구를 정의했으니 에이전트를 생성할 수 있습니다. OpenAI Functions 에이전트를 사용할 것입니다.

먼저, 에이전트가 활용할 LLM을 정의합니다.


`ChatOpenAI` 클래스는 `langchain_openai` 모듈에서 가져온 것으로, OpenAI의 언어 모델을 활용하여 대화형 AI를 구현할 수 있게 해줍니다.

이 예제에서는 `gpt-4-turbo-preview` 모델을 사용하며, `temperature` 매개변수를 `0`으로 설정하여 예측의 변동성을 최소화합니다.


In [10]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 클래스를 langchain_openai 모듈에서 가져옵니다.
gpt = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)

다음으로, 에이전트를 안내하기 위해 사용하고 싶은 프롬프트를 선택합니다.

이 프롬프트의 내용을 보고 싶고 LangSmith에 접근할 수 있다면, 다음으로 이동하세요:

다운로드 받아 활용할 프롬프트: https://smith.langchain.com/hub/hwchase17/openai-functions-agent


`hub.pull()` 함수를 사용하여 리소스를 가져온 후, `prompt.messages`를 통해 해당 리소스의 메시지에 접근합니다.


In [11]:
from langchain import hub

# hub에서 prompt를 가져옵니다 - 이 부분을 수정할 수 있습니다!
prompt = hub.pull("hwchase17/openai-functions-agent")

# prompt 의 messages를 출력합니다.
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

이제 LLM, 프롬프트 및 도구로 에이전트를 초기화할 수 있습니다. 에이전트는 입력을 받아 어떤 `Action` 을 취할지 결정하는 역할을 합니다.

중요한 것은 에이전트가 `Action` 들을 실행하지 않는다는 것입니다 - 그것은 AgentExecutor(다음 단계)에 의해 수행됩니다.


In [12]:
from langchain.agents import create_openai_functions_agent

# OpenAI 함수 기반 에이전트를 생성합니다.
# llm, tools, prompt를 인자로 사용합니다.
agent = create_openai_functions_agent(gpt, tools, prompt)

마지막으로, 우리는 에이전트(`agent`)를 AgentExecutor 내부의 도구들과 결합합니다(이는 반복적으로 에이전트를 호출하고 도구들을 실행할 것입니다).


이 코드는 `langchain.agents` 모듈에서 `AgentExecutor` 클래스를 가져와 인스턴스를 생성합니다. 생성 시, `agent`, `tools` 객체를 인자로 전달하고, `verbose=True`를 설정하여 상세한 로그 출력을 활성화합니다. `AgentExecutor`는 주어진 에이전트와 도구들을 사용하여 작업을 실행하는 역할을 합니다.


In [13]:
from langchain.agents import AgentExecutor

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정합니다.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## 에이전트 실행하기

이제 몇 가지 질의에 대해 에이전트를 실행할 수 있습니다!

현재 이러한 모든 질의는 **상태(Stateless) 가 없는** 질의입니다(이전 상호작용을 기억하지 않습니다).


`agent_executor` 객체의 `invoke` 메소드는 딕셔너리 형태의 인자를 받아 처리합니다. 이 예제에서는 `input` 키에 `hi!` 값을 할당한 딕셔너리를 인자로 전달하고 있습니다. 이는 일반적으로 AI 에이전트, 함수 실행기, 또는 명령 처리기 등의 객체에서 입력을 처리하기 위해 사용됩니다.


In [14]:
# 검색 결과를 요청 후 질문에 대한 답변을 출력합니다.
response = agent_executor.invoke(
    {"input": "판교 몽중헌 전화번호를 검색하여 결과를 알려주세요."}
)
print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '판교 몽중헌 전화번호'}`


[{'url': 'http://www.mongjungheon.co.kr/pangyo.jsp', 'content': '남기는 백주\n500ml 75,000\n농향형 34% 연태지방의 수수를 원료로 만든 향이 좋고 부드러운 백주\n500ml 65,000\n250ml 38,000\n125ml 28,000\nCHAMPAGNE · SPARKLING WINE\n350,000\n250,000\n160,000\n90,000\n70,000\nWHITE WINE\n170,000\n110,000\n95,000\n90,000\n85,000\n75,000\nRED WINE\n380,000\n180,000\n150,000\n130,000\n120,000\n100,000\n95,000\n90,000\n80,000\n75,000\n70,000\nGLASS WINE\n15,000\n15,000\nDISTILLED LIQUOR\n375ml 35,000\n375ml 35,000\nBEER\n320ml 11,000\n640ml 14,000\n580ml 11,000\n600ml 12,000\n610ml 12,000\n330ml 8,000\n330ml 8,000\nSOFT DRINK\n7,000\n7,000\n5,000\n5,000\n4,500\n8,000\nCOFFEE\nhot 5,000\nIced 6,000\nhot 6,000\nIced 7,000\nhot 5,000\nDim Sum Special Course\nLUNCH - 70,000 (1人)\nDINNER - 90,000 (1人)\n Chef-Recommended\nSpicy Dish\n대만\n금문향형 58% 최상의 수질과 기후를 가진 금문도에서 자란 수수를 엄선하여 제조한 술로, 부드럽고 감미로운 맛이 우수한 백주\n600ml 180,00

`agent_executor` 객체의 `invoke` 메소드를 사용하여, 질문을 입력으로 제공합니다.


In [15]:
response = agent_executor.invoke(
    {
        "input": "YouTube 2024년부터 AI 생성콘텐츠 표시 의무화에 대한 내용을 PDF 문서에서 알려줘"
    }
)
print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...

Invoking: `pdf_search` with `{'query': 'YouTube 2024년 AI 생성콘텐츠 표시 의무화'}`


SPRi AI Brief |  
2023-12 월호
14유튜브 , 2024년부터 AI 생성 콘텐츠 표시 의무화 
n유튜브가 몇 달 안에 생성 AI를 사용한 콘텐츠에 AI 라벨 표시를 의무화하기로 했으며 , 이를 
준수하지 않는 콘텐츠는 삭제하고 크리에이터에 대한 수익 배분도 중단할 수 있다고 설명
n유튜브는 AI 생성 콘텐츠가 신원 파악이 가능한 개인을 모방한 경우 개인정보 침해 신고 
절차에 따라 콘텐츠 삭제 요청도 받을 계획KEY Contents
£유튜브 , 생성 AI 콘텐츠에 AI 라벨 표시 안 하면 콘텐츠 삭제
n유튜브가 2023 년 11월 14일 공식 블로그를 통해 몇 달 안에 생성 AI를 사용한 콘텐츠에 AI 
라벨을 표시하는 새로운 규칙을 시행한다고 발표 
∙실제로 일어나지 않은 사건을 사실적으로 묘사하거나 실제로 하지 않은 말이나 행동을 보여주는 콘텐츠와 
같이 AI 도구를 사용해 사실적으로 변경되거나 합성된 콘텐츠에는 AI 라벨을 표시 필요
∙유튜브는 이러한 규칙이 선거나 분쟁 상황, 공중 보건, 공직자 관련 문제와 같이 민감한 주제를 다루는 
콘텐츠에서 특히 중요하다고 강조했으며 , 크리에이터가 AI로 제작한 콘텐츠에 AI 라벨을 표시하지 않으면 
해당 콘텐츠는 삭제되고 광고 수익을 배분하는 유튜브 파트너 프로그램도 정지될 수 있음
∙유튜브는 두 가지 방식으로 AI를 이용한 콘텐츠의 변경이나 합성 여부를 시청자에게 전달할 계획으로 
동영상 설명 패널에 라벨을 표시하는 방식이 기본이며 , 민감한 주제를 다루는 특정 유형의 콘텐츠는 동영상 
플레이어에 더욱 눈에 띄는 라벨을 적용 
∙유튜브는 커뮤니티 정책에 위반되는 일부 합성 콘텐츠에 대해서는 라벨 지정 여부와 관계없이 삭제할 
방침으로 , 가령 사실적인 폭력을 보여주는 합성 동영상이 시청자

## EEVE Korean (LM Studio) - 동작 X


In [16]:
from langchain_openai import ChatOpenAI

In [17]:
eeve = ChatOpenAI(
    model="teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    temperature=0.1,
)

In [18]:
eeve_agent = create_openai_functions_agent(eeve, tools, prompt)

In [19]:
eeve_agent_executor = AgentExecutor(
    agent=eeve_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

In [20]:
# 검색 결과를 요청 후 질문에 대한 답변을 출력합니다.
response = eeve_agent_executor.invoke(
    {"input": "판교 몽중헌의 전화번호를 검색하여 결과를 알려주세요."}
)
print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...
<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
판교 몽중헌의 전화번호를 검색하여 결과를 알려주세요.<|im_end|>
<|im_start|>assistant
판교 몽중헌은 '판교 몽촌마을'로도 알려져 있으며, 주거 단지로 전화번호가 따로 존재하지 않습니다. 그러나 해당 지역의 관리사무소나 부동산 중개업소에 연락하시면 거주민이나 관련 정보를 안내받을 수 있을 것입니다. 판교 몽촌마을의 위치는 경기도 성남시 분당구 삼평동 620-1번지입니다.

> Finished chain.
답변: <|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
판교 몽중헌의 전화번호를 검색하여 결과를 알려주세요.<|im_end|>
<|im_start|>assistant
판교 몽중헌은 '판교 몽촌마을'로도 알려져 있으며, 주거 단지로 전화번호가 따로 존재하지 않습니다. 그러나 해당 지역의 관리사무소나 부동산 중개업소에 연락하시면 거주민이나 관련 정보를 안내받을 수 있을 것입니다. 판교 몽촌마을의 위치는 경기도 성남시 분당구 삼평동 620-1번지입니다.


In [21]:
# 검색 결과를 요청 후 질문에 대한 답변을 출력합니다.
response = eeve_agent_executor.invoke(
    {"input": "2024년 5월 1일의 서울 날씨를 검색해서 알려줘."}
)
print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...
<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
2024년 5월 1일의 서울 날씨를 검색해서 알려줘.<|im_end|>
<|im_start|>assistant
2024년 5월 1일, 서울의 예상 기온은 최고기온이 섭씨 23도(화씨 73.4도), 최저기온이 섭씨 16도(화씨 60.8도)로 비교적 온화하고 쾌적한 날씨를 보일 것으로 예상됩니다. 이 날에는 대체로 구름이 조금 있는 가운데, 강수 확률은 약 20%입니다.

> Finished chain.
답변: <|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
2024년 5월 1일의 서울 날씨를 검색해서 알려줘.<|im_end|>
<|im_start|>assistant
2024년 5월 1일, 서울의 예상 기온은 최고기온이 섭씨 23도(화씨 73.4도), 최저기온이 섭씨 16도(화씨 60.8도)로 비교적 온화하고 쾌적한 날씨를 보일 것으로 예상됩니다. 이 날에는 대체로 구름이 조금 있는 가운데, 강수 확률은 약 20%입니다.


## llama 3 (JSON Agent)

- 링크: https://github.com/sionic-ai/xionic-ko-llama-3-70b


In [22]:
llama3 = ChatOpenAI(
    base_url="http://sionic.chat:8001/v1",
    api_key="934c4bbc-c384-4bea-af82-1450d7f8128d",
    model="xionic-ko-llama-3-70b",
    temperature=0.1,
)

In [23]:
# Use Prompt for JSON Agent
json_prompt = hub.pull("hwchase17/react-chat-json")

In [25]:
from langchain.agents import create_json_chat_agent

llama3_agent = create_json_chat_agent(llama3, tools, json_prompt)

In [26]:
from langchain.agents import AgentExecutor

llama3_agent_executor = AgentExecutor(
    agent=llama3_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
)

In [27]:
# 검색 결과를 요청 후 질문에 대한 답변을 출력합니다.
response = llama3_agent_executor.invoke(
    {"input": "판교 몽중헌 전화번호를 검색하여 결과를 알려주세요."}
)
print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...
```
{
    "action": "tavily_search_results_json",
    "action_input": "몽중헌 판교 전화번호"
}
```[{'url': 'https://app.catchtable.co.kr/ct/shop/mongjungheon_pg', 'content': '중식 | 판교 몽중헌 판교점 ... - 판교 테크원 타워 건물 내 지하 주차장을 이용 부탁드리며 , 2시간 이용가능 합니다. ... 전화번호 031-601-7574. 잘못된 정보가 있나요? 수정이 필요하거나 폐점한 매장이라면 알려주세요! 저장 1,225. 예약하기 ...'}, {'url': 'http://www.mongjungheon.co.kr/pangyo.jsp', 'content': '남기는 백주\n500ml 75,000\n농향형 34% 연태지방의 수수를 원료로 만든 향이 좋고 부드러운 백주\n500ml 65,000\n250ml 38,000\n125ml 28,000\nCHAMPAGNE · SPARKLING WINE\n350,000\n250,000\n160,000\n90,000\n70,000\nWHITE WINE\n170,000\n110,000\n95,000\n90,000\n85,000\n75,000\nRED WINE\n380,000\n180,000\n150,000\n130,000\n120,000\n100,000\n95,000\n90,000\n80,000\n75,000\n70,000\nGLASS WINE\n15,000\n15,000\nDISTILLED LIQUOR\n375ml 35,000\n375ml 35,000\nBEER\n320ml 11,000\n640ml 14,000\n580ml 11,000\n600ml 12,000\n610ml 12,000\n330ml 8,000\n330ml 8,000\nSOFT DRINK\n7,000\n7,000\n5,000\n5,000\n4,500\n8,000\nCOFF

In [28]:
# 검색 결과를 요청 후 질문에 대한 답변을 출력합니다.
response = llama3_agent_executor.invoke(
    {
        "input": "YouTube 2024년부터 AI 생성콘텐츠 표시 의무화에 대한 내용을 PDF 문서에서 알려줘"
    }
)
print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...
```
{
    "action": "pdf_search",
    "action_input": "YouTube 2024년부터 AI 생성콘텐츠 표시 의무화"
}
```SPRi AI Brief |  
2023-12 월호
14유튜브 , 2024년부터 AI 생성 콘텐츠 표시 의무화 
n유튜브가 몇 달 안에 생성 AI를 사용한 콘텐츠에 AI 라벨 표시를 의무화하기로 했으며 , 이를 
준수하지 않는 콘텐츠는 삭제하고 크리에이터에 대한 수익 배분도 중단할 수 있다고 설명
n유튜브는 AI 생성 콘텐츠가 신원 파악이 가능한 개인을 모방한 경우 개인정보 침해 신고 
절차에 따라 콘텐츠 삭제 요청도 받을 계획KEY Contents
£유튜브 , 생성 AI 콘텐츠에 AI 라벨 표시 안 하면 콘텐츠 삭제
n유튜브가 2023 년 11월 14일 공식 블로그를 통해 몇 달 안에 생성 AI를 사용한 콘텐츠에 AI 
라벨을 표시하는 새로운 규칙을 시행한다고 발표 
∙실제로 일어나지 않은 사건을 사실적으로 묘사하거나 실제로 하지 않은 말이나 행동을 보여주는 콘텐츠와 
같이 AI 도구를 사용해 사실적으로 변경되거나 합성된 콘텐츠에는 AI 라벨을 표시 필요
∙유튜브는 이러한 규칙이 선거나 분쟁 상황, 공중 보건, 공직자 관련 문제와 같이 민감한 주제를 다루는 
콘텐츠에서 특히 중요하다고 강조했으며 , 크리에이터가 AI로 제작한 콘텐츠에 AI 라벨을 표시하지 않으면 
해당 콘텐츠는 삭제되고 광고 수익을 배분하는 유튜브 파트너 프로그램도 정지될 수 있음
∙유튜브는 두 가지 방식으로 AI를 이용한 콘텐츠의 변경이나 합성 여부를 시청자에게 전달할 계획으로 
동영상 설명 패널에 라벨을 표시하는 방식이 기본이며 , 민감한 주제를 다루는 특정 유형의 콘텐츠는 동영상 
플레이어에 더욱 눈에 띄는 라벨을 적용 
∙유튜브는 커뮤니티 정책에 위반되는 일부 합성 콘텐츠에 대해서는 라벨 지정 여부와 관계없이 삭제할 
방침으로 , 가령 사실적인 폭력을

## 한글 답변 처리


In [29]:
prompt_ko = hub.pull("teddynote/react-chat-json-korean")

In [30]:
from langchain.agents import create_json_chat_agent

llama3_agent_ko = create_json_chat_agent(llama3, tools, prompt_ko)

In [31]:
from langchain.agents import AgentExecutor

llama3_agent_executor = AgentExecutor(
    agent=llama3_agent_ko,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
)

In [32]:
# 검색 결과를 요청 후 질문에 대한 답변을 출력합니다.
response = llama3_agent_executor.invoke(
    {"input": "판교 몽중헌 전화번호를 검색하여 결과를 알려주세요."}
)
print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...
```
{
    "action": "tavily_search_results_json",
    "action_input": "판교 몽중헌 전화번호"
}
```[{'url': 'https://app.catchtable.co.kr/ct/shop/mongjungheon_pg', 'content': '중식 | 판교 몽중헌 판교점 ... - 판교 테크원 타워 건물 내 지하 주차장을 이용 부탁드리며 , 2시간 이용가능 합니다. ... 전화번호 031-601-7574. 잘못된 정보가 있나요? 수정이 필요하거나 폐점한 매장이라면 알려주세요! 저장 1,225. 예약하기 ...'}, {'url': 'https://blog.naver.com/PostView.naver?blogId=dalgi6736&logNo=223419347824&noTrackingCode=true', 'content': '📞 전화번호: 031-601-7574. 🌐 홈페이지: 몽중헌 공식 홈페이지 오늘은 가족모임 장소로 추천하는 분당 중식당 몽중헌을 소개해봅니다. 저는 가족모임이 있어서 룸이있는 식당을 찾다가 판교에 몽중헌이 있는걸 알고 예약을 한달전에 진행했어요'}, {'url': 'http://www.mongjungheon.co.kr/', 'content': '*2인 이상 주문 시 가능합니다.\n냉채류\n탕류\n잡품류\n바닷가재·새우류\n쇠고기류\n돼지고기류\n닭고기류\n구이류\n채소·두부류\n면류\n밥류\nSteamed Dim Sum\n조리에 사용되는 닭육수용 닭은 국내산만 사용합니다.\n Chef-Recommended\nSpicy Dish\nDim Sum Special Course\nLUNCH - 75,000 (1人)\nDINNER - 95,000 (1人)\n조리에 사용되는 닭육수용 닭은 국내산만 사용합니다.\n 할인 제공 종료\n기프트 카드\n몽중헌 브랜드 전용 기프트 카드\nChef ’s Special Course / All Da

In [33]:
# 검색 결과를 요청 후 질문에 대한 답변을 출력합니다.
response = llama3_agent_executor.invoke(
    {
        "input": "YouTube 2024년부터 AI 생성콘텐츠 표시 의무화에 대한 내용을 PDF 문서에서 알려줘"
    }
)
print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...
```
{
    "action": "pdf_search",
    "action_input": "YouTube 2024년부터 AI 생성콘텐츠 표시 의무화"
}
```SPRi AI Brief |  
2023-12 월호
14유튜브 , 2024년부터 AI 생성 콘텐츠 표시 의무화 
n유튜브가 몇 달 안에 생성 AI를 사용한 콘텐츠에 AI 라벨 표시를 의무화하기로 했으며 , 이를 
준수하지 않는 콘텐츠는 삭제하고 크리에이터에 대한 수익 배분도 중단할 수 있다고 설명
n유튜브는 AI 생성 콘텐츠가 신원 파악이 가능한 개인을 모방한 경우 개인정보 침해 신고 
절차에 따라 콘텐츠 삭제 요청도 받을 계획KEY Contents
£유튜브 , 생성 AI 콘텐츠에 AI 라벨 표시 안 하면 콘텐츠 삭제
n유튜브가 2023 년 11월 14일 공식 블로그를 통해 몇 달 안에 생성 AI를 사용한 콘텐츠에 AI 
라벨을 표시하는 새로운 규칙을 시행한다고 발표 
∙실제로 일어나지 않은 사건을 사실적으로 묘사하거나 실제로 하지 않은 말이나 행동을 보여주는 콘텐츠와 
같이 AI 도구를 사용해 사실적으로 변경되거나 합성된 콘텐츠에는 AI 라벨을 표시 필요
∙유튜브는 이러한 규칙이 선거나 분쟁 상황, 공중 보건, 공직자 관련 문제와 같이 민감한 주제를 다루는 
콘텐츠에서 특히 중요하다고 강조했으며 , 크리에이터가 AI로 제작한 콘텐츠에 AI 라벨을 표시하지 않으면 
해당 콘텐츠는 삭제되고 광고 수익을 배분하는 유튜브 파트너 프로그램도 정지될 수 있음
∙유튜브는 두 가지 방식으로 AI를 이용한 콘텐츠의 변경이나 합성 여부를 시청자에게 전달할 계획으로 
동영상 설명 패널에 라벨을 표시하는 방식이 기본이며 , 민감한 주제를 다루는 특정 유형의 콘텐츠는 동영상 
플레이어에 더욱 눈에 띄는 라벨을 적용 
∙유튜브는 커뮤니티 정책에 위반되는 일부 합성 콘텐츠에 대해서는 라벨 지정 여부와 관계없이 삭제할 
방침으로 , 가령 사실적인 폭력을